In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
from unidecode import unidecode

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [2]:
with open('json0929/permanentes.json') as f:
    data = json.load(f)

In [3]:
def filter_list_of_dicts(list_of_dicts, property_name, property_value):
    """
    This function filters a list of dictionaries by a given property name and value using the filter() function.
    :param list_of_dicts: list of dictionaries to be filtered
    :param property_name: name of the property to filter by
    :param property_value: value of the property to filter by
    :return: a list of dictionaries that match the filter criteria
    """
    return filter(lambda d: d.get(property_name) == property_value, list_of_dicts)


In [4]:
def get_item_by_key(key: int) -> dict:
    """
    This function returns a dictionary item from a list of dictionaries based on a given key.
    :param key: key to filter by
    :return: a dictionary item that matches the key
    """
    return next(filter_list_of_dicts(data, 'key', key), None)

In [16]:
def batch_write_update_medidas(collection_name: str, id_list: list, medidas: str):
    batch = db.batch() 
    collection_ref = db.collection(collection_name)
    k = 1
    for i in range(0, len(id_list), 500):
        chunk = id_list[i:i+500]
        for idx in chunk: 
            doc_ref = collection_ref.document(str(idx))
            batch.update(doc_ref, {'detalhes.medidas': medidas})
        print(f"{collection_name}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [5]:
cadeira = get_item_by_key(337921)
cadeira['detalhes']

{'patrimonio': 543858,
 'n_controle': None,
 'cod_barras': 337921,
 'n_serie': None,
 'descricao': 'CADEIRA ESCOLAR UNIVERSITÁRIA - CADEIRA ESCOLAR COM PRANCHETA FIXA EM MDF, ASSENTO E ENCOSTO EM COMPENSADO E ESPUMA, REVESTIDO NA COR AZUL ROYAL. ESTRUTURA CONFECCIONADA EM TUBO DE AÇO PINTADO EM PRETO.',
 'conservacao': 'EM USO',
 'incorporacao': '12/02/2018',
 'transf_local': None,
 'valor': 140,
 'situacao': 'Ativo',
 'peso': '',
 'medidas': ''}

In [ ]:
1=2

# Cadeiras

## Escolar

In [36]:
cadeiras_escolar_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', "CADEIRA ESCOLAR UNIVERSITÁRIA") ]
    

In [37]:
batch_write_update_medidas("items", cadeiras_escolar_ids, '95 x 80 x 56 (AxCxL)')

items: chunk 1 (500 documents)
items: chunk 2 (474 documents)


## Cadeira com braços

In [39]:
get_item_by_key(310264)["short_descricao"]
# 'CADEIRA ESCRITÓRIO FIXA COM BRAÇO'

'CADEIRA ESCRITÓRIO FIXA COM BRAÇO'

In [42]:
cadeiras_com_braço_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(310264)["short_descricao"])]


In [46]:
batch_write_update_medidas("items", cadeiras_com_braço_ids, '96 x 59 x 63 (AxCxL)')

items: chunk 1 (98 documents)


## Cadeira sem braços

In [56]:
get_item_by_key(347857)["short_descricao"]


'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS'

In [57]:
cadeiras_sem_braço_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(347857)["short_descricao"])]


In [58]:
len(cadeiras_sem_braço_ids)

232

In [59]:
set([ x['detalhes']['descricao'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(347857)["short_descricao"])])

{'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM,TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM.',
 'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM,TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM. ',
 'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - MARCA: FLEXFORM; MODELO: DOTY/CL.DOTY.C.EF. ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM, TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM.'}

In [60]:
batch_write_update_medidas("items", cadeiras_sem_braço_ids, '87 x 55 x 51 (AxCxL)')

items: chunk 1 (232 documents)


## Mesa reta 4 pés

In [53]:
get_item_by_key(363625)["short_descricao"]

'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM '

In [67]:
mesa_reta_4 = filter_list_of_dicts(data, 'short_descricao', get_item_by_key(363625)["short_descricao"])

In [60]:
set([ x['detalhes']['descricao'] for x in  mesa_reta_4])

{'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MARCA: TECNO2000, MODELO: BREMEN. MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CLARO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MARCA: TECNO2000; MODELO: BREMEN. MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CLARO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CRISTAL, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR OVO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.'}

In [68]:
mesa_reta_4_ids = [ x['key'] for x in  mesa_reta_4]

In [70]:
batch_write_update_medidas("items", mesa_reta_4_ids, '75 x 60 x 80 (AxCxL)')

items: chunk 1 (88 documents)


## Mesa retangular para computador

In [14]:
get_item_by_key(327223)["detalhes"]["descricao"]

mesa_ret_comp = list(filter_list_of_dicts(data, 'short_descricao', get_item_by_key(327223)["short_descricao"]))

In [20]:
desc_mesa_ret_comp = [x["detalhes"]["descricao"] for x in mesa_ret_comp]

In [40]:
desc_mesa_ret_comp

mesa_ret_comp_ids = [ x['key'] for x in mesa_ret_comp if x["detalhes"]["descricao"].find("160 X 77") > 0 ]


In [44]:
batch_write_update_medidas('items', mesa_ret_comp_ids, '75 x 160 x 77 (AxCxL)')

items: chunk 1 (95 documents)


## ARMÁRIO ALTO PARA ESCRITÓRIO

In [51]:
armario_algo_escrit = get_item_by_key(322722)

In [52]:
armarios_algo_escrit = [ x  for x in  filter_list_of_dicts(data, 'short_descricao', armario_algo_escrit["short_descricao"]) ]

In [53]:
set([ x['detalhes']['descricao'] for x in  armarios_algo_escrit])

{'ARMÁRIO ALTO PARA ESCRITÓRIO - ARMÁRIO ALTO PARA ESCRITÓRIO EM MADEIRA MDF REVESTIDO NA COR CINZA CRISTAL, COM DUAS PORTAS DE ABRIR, CHAVES, 3 PRATELEIRAS REGULÁVEIS. DIMENSÕES 160X80X50 CM.',
 'ARMÁRIO ALTO PARA ESCRITÓRIO - ARMÁRIO ALTO PARA ESCRITÓRIO EM MADEIRA MDF REVESTIDO NA COR CINZA CRISTAL, COM DUAS PORTAS DE ABRIR, CHAVES, 4 PRATELEIRAS REGULÁVEIS. DIMENSÕES 170X90X48 CM.',
 'ARMÁRIO ALTO PARA ESCRITÓRIO - ARMÁRIO ALTO PARA ESCRITÓRIO EM MADEIRA MDF REVESTIDO NA COR OVO, COM DUAS PORTAS DE ABRIR, CHAVES, PRATELEIRAS REGULÁVEIS. DIMENSÕES 160X90X45 CM.'}

In [54]:
ids_armarios_algo_escrit = [ x['key'] for x in  armarios_algo_escrit  ]

In [55]:
batch_write_update_medidas('items', ids_armarios_algo_escrit, '171 x 49 x 91 (AxCxL)')

items: chunk 1 (90 documents)


##  CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS

In [7]:
cadeira_dialogo_sembracos = get_item_by_key(359621)

In [8]:
cadeiras_dialogo = list(filter_list_of_dicts(data, 'short_descricao', cadeira_dialogo_sembracos["short_descricao"]))

In [13]:
ids_cadeiras_dialogo = [ x['key'] for x in cadeiras_dialogo if x["detalhes"]["descricao"] == cadeira_dialogo_sembracos["detalhes"]["descricao"] ]

In [17]:
batch_write_update_medidas('items', ids_cadeiras_dialogo, '87 x 50 x 53 (AxCxL)')

items: chunk 1 (119 documents)


## CADEIRA FIXA 04 PÉS SEM BRAÇOS

In [18]:
cadeira_fixa_4 = get_item_by_key(353087)

In [19]:
cadeiras_fixa_4 = list(filter_list_of_dicts(data, 'short_descricao', cadeira_fixa_4["short_descricao"]))

In [22]:
ids_cadeiras_fixa_4 = [ x['key'] for x in cadeiras_fixa_4 ]

In [23]:
batch_write_update_medidas('items', ids_cadeiras_fixa_4, '78 x 54 x 54 (AxCxL)')

items: chunk 1 (162 documents)


## MESA PARA MICROCOMPUTADOR

In [24]:
mesa_compiuter = get_item_by_key(317156)

In [25]:
mesas_compiuter = list(filter_list_of_dicts(data, 'short_descricao', mesa_compiuter["short_descricao"]))

In [26]:
set([ x['detalhes']['descricao'] for x in  mesas_compiuter])

{'MESA PARA MICROCOMPUTADOR - MESA PARA MICROCOMPUTADOR EM MADEIRA MDF REVISTIDA NA COR CINZA COM SUPORTE RETRÁTIL PARA TECLADO. \nMEDIDAS: 75 X 80 X 70 CM, MEDIDA DO SUPORTE RETRÁTIL PARA TECLADO : 58 X 33 CM.',
 'MESA PARA MICROCOMPUTADOR - MESA PARA MICROCOMPUTADOR EM MADEIRA MDF REVISTIDA NA COR CINZA. MEDIDAS: 75 X 80 X 70 CM.',
 'MESA PARA MICROCOMPUTADOR - REVESTIDA EM ARVOPLAC, NA COR CASCA DE OVO, C/ESTRUTURA DE FERRO NA COR OVO, C/REBAIXO P/TECLADO RETRATIL, MED. 110X60X70CM.'}

In [27]:
len(mesas_compiuter)

81

In [28]:
ids_mesas_compiuter = [ x['key'] for x in mesas_compiuter ] 

In [29]:
batch_write_update_medidas('items', ids_mesas_compiuter, '75 x 70 x 80 (AxCxL)')

items: chunk 1 (81 documents)


## BANQUETA  PARA LABORATÓRIO

In [30]:
banqueta_lab = get_item_by_key(352801)

In [31]:
banquetas_lab = list(filter_list_of_dicts(data, 'short_descricao', banqueta_lab["short_descricao"]))

In [32]:
len(banquetas_lab)

161

In [33]:
set([ x['detalhes']['descricao'] for x in  banquetas_lab])

{'BANQUETA  PARA LABORATÓRIO - BANQUETA COM QUATRO PERNAS, COM ASSENTO REDONDO, EM MADEIRA ANGELIM PARA LABORATÓRIO, DIMENSÕES: 70 X 30CM.',
 'BANQUETA  PARA LABORATÓRIO - BANQUETA DE MADEIRA COM QUATRO PERNAS, NA COR BRANCA. COM 60 CM DE ALTURA E ASSENTO MEDINDO 30 CM DE DIÂMETRO. ',
 'BANQUETA  PARA LABORATÓRIO - BANQUETA DE MADEIRA COM QUATRO PERNAS, NA COR BRANCA. COM 70 CM DE ALTURA E ASSENTO MEDINDO 30 CM DE DIÂMETRO.',
 'BANQUETA  PARA LABORATÓRIO - BANQUETA EM MADEIRA ANGELIM PARA LABORATÓRIO, DIMENSÕES: 70 X 30CM.'}

In [34]:
ids_banquetas_lab = [ x['key'] for x in banquetas_lab ]

In [36]:
batch_write_update_medidas('items', ids_banquetas_lab, '70 x 37 x 37 (AxCxL)')

items: chunk 1 (161 documents)


## CADEIRA GIRATÓRIA ESPALDAR MÉDIO

In [37]:
cadeira_gira = get_item_by_key(347771)

In [38]:
cadeiras_gira = list(filter_list_of_dicts(data, 'short_descricao', cadeira_gira["short_descricao"]))

In [39]:
len(cadeiras_gira)

44

In [40]:
set([ x['detalhes']['descricao'] for x in  cadeiras_gira])

{'CADEIRA GIRATÓRIA ESPALDAR MÉDIO - MARCA: TOK; MODELO: 93EPSY GIR S/BR. EM TECIDO E ESTRUTURA DE AÇO NA COR PRETA. COM 5 RODÍZIOS. SEM APOIOS PARA OS BRAÇOS, COM ALTURA REGULÁVEL.'}

In [41]:
ids_cadeiras_gira = [ x['key'] for x in cadeiras_gira ]

In [42]:
batch_write_update_medidas('items', ids_cadeiras_gira, '95 x 60 x 46 (AxCxL)')

items: chunk 1 (44 documents)


## QUADRO

In [61]:
quadro = get_item_by_key(327526)

In [62]:
quadros = list(filter_list_of_dicts(data, 'short_descricao', quadro["short_descricao"]))

In [63]:
set([ x['detalhes']['descricao'] for x in  quadros])

{'QUADRO - BRANCO QUADRICULADO, EM ESTRUTURA DE ALUMINIO, P/USO PINCEL ATOMICO, C/SUPORTE P/ APAGADOR, SUPORTE P/FIXAR NA PAREDE, MED. 300X120CM',
 'QUADRO - INTERATIVO, MARCA PROMETHEAN, MOD. ACTIVBOARD, C/SENSOR, NA COR BRANCA/CINZA, C/SUPORTE P/FIXAR NA PAREDE, C/ACESSORIOS, C/POTENCIA DE 6V, MED. 170X124CM.',
 'QUADRO - VERDE, EM ESTRUTURA DE ALUMINIO, P/USO DE GIZ, C/SUPORTE P/ APAGADOR, SUPORTE P/FIXAR NA PAREDE, MED. 300X120CM'}

In [64]:
quadros_ids = [ x['key'] for x in quadros if x["detalhes"]["descricao"].find("300X120") > 0 ]

In [66]:
len(quadros_ids)

33

In [67]:
batch_write_update_medidas('items', quadros_ids, '300 x 120 x 4 (AxCxL)')

items: chunk 1 (33 documents)


## MESA PARA ESCRITÓRIO II

In [68]:
mesa_II = get_item_by_key(359999)

In [69]:
mesas_II = list(filter_list_of_dicts(data, 'short_descricao', mesa_II["short_descricao"]))

In [71]:
len(mesas_II)

76

In [73]:
ids_mesas_II = [ x['key'] for x in mesas_II  if x["detalhes"]["descricao"] == mesa_II["detalhes"]["descricao"] ]

In [75]:
batch_write_update_medidas('items', ids_mesas_II, '75 x 140 x 65 (AxCxL)')

items: chunk 1 (30 documents)


## COMPUTADOR DESKTOP (SFF) DE USO GERAL

### DELL OPTIPLEX 5050 

In [99]:
computadorSSF1 = get_item_by_key(338116)

In [100]:
computadoresSSF1 = list(filter_list_of_dicts(data, 'short_descricao', computadorSSF1["short_descricao"]))

In [101]:
compuiuters_ids1 = [ x['key'] for x in computadoresSSF  if x["detalhes"]["descricao"] == computadorSSF1["detalhes"]["descricao"] ]

In [82]:
batch_write_update_medidas('items', compuiuters_ids1, '10 x 29 x 29 (AxCxL)')

items: chunk 1 (27 documents)


### HP PRODESK 600

In [103]:
computadorSSF2 = get_item_by_key(337701)

In [105]:
computadoresSSF2 = list(filter_list_of_dicts(data, 'short_descricao', computadorSSF2["short_descricao"]))

In [106]:
compuiuters_ids2 = [ x['key'] for x in computadoresSSF2  if x["detalhes"]["descricao"] == computadorSSF2["detalhes"]["descricao"] ]

In [107]:
batch_write_update_medidas('items', compuiuters_ids2, '10 x 38 x 34 (AxCxL)')

items: chunk 1 (22 documents)


### DELL OPTIPLEX 3080

In [113]:
computadorSSF3 = get_item_by_key(364186)

In [114]:
computadoresSSF3 = list(filter_list_of_dicts(data, 'short_descricao', computadorSSF3["short_descricao"]))

In [115]:
computadoresSSF3ids = [ x['key'] for x in computadoresSSF3  if x["detalhes"]["descricao"] == computadorSSF3["detalhes"]["descricao"] ]

In [116]:
batch_write_update_medidas('items', computadoresSSF3ids, '5 x 18 x 19 (AxCxL)')

items: chunk 1 (18 documents)


## MONITOR P/ESTACAO DE TRABALHO

In [90]:
monitor_estacao = get_item_by_key(387062)
monitor_estacao

{'responsavel': 'Cristiano da Silva Teixeira',
 'ambiente': 'B021',
 'edificio': 'BNU03 - Bloco Acadêmico B',
 'detalhes': {'patrimonio': 1001703,
  'n_controle': None,
  'cod_barras': 387062,
  'n_serie': '3SDYB03',
  'descricao': "MONITOR P/ESTACAO DE TRABALHO - MONITOR DELL DE 21,5'', MODELO P2219H, ACOMPANHA CABOS E MANUAL, 220V.",
  'conservacao': 'EM USO',
  'incorporacao': '22/10/2020',
  'transf_local': '16/11/2022',
  'valor': 396.21,
  'situacao': 'Ativo',
  'peso': '',
  'medidas': ''},
 'short_descricao': 'MONITOR P/ESTACAO DE TRABALHO',
 'origem': 'B021',
 'key': 387062,
 'meta': {'inteiro': True,
  'updated': '29/09/2023 10:51:30',
  'partes': [],
  'volumado': False,
  'volume': None},
 'tipo': 'Permanente'}

In [84]:
monitores_estacao = list(filter_list_of_dicts(data, 'short_descricao', monitor_estacao["short_descricao"]))

In [91]:
monitores_estacao_ids = [ x['key'] for x in monitores_estacao ]

In [89]:
set([ x['detalhes']['descricao'] for x in  monitores_estacao])

{'MONITOR P/ESTACAO DE TRABALHO - MARCA: DELL; MODELO P2422H, EM LCD 23,8 POLEGADAS, COR PRETA, ACOMPANHA CD, CABO. BIVOLT.',
 "MONITOR P/ESTACAO DE TRABALHO - MONITOR DELL DE 21,5'', MODELO P2219H, ACOMPANHA CABOS E MANUAL, 220V.",
 "MONITOR P/ESTACAO DE TRABALHO - MONITOR DELL DE 21,5'', MODELO P2419H, ACOMPANHA CABOS E MANUAL, 220V.",
 "MONITOR P/ESTACAO DE TRABALHO - MONITOR DELL DE 23,8'', MODELO P2422H, ACOMPANHA CABOS E MANUAL, 220V."}

In [92]:
batch_write_update_medidas('items', monitores_estacao_ids, '50 x 54 x 18 (AxCxL)')

items: chunk 1 (74 documents)


## CADEIRA ESCOLAR

In [93]:
cadeira_escolar = get_item_by_key(139450)

In [94]:
cadeiras_escolares = list(filter_list_of_dicts(data, 'short_descricao', cadeira_escolar["short_descricao"]))

In [96]:
set([ x['detalhes']['descricao'] for x in  cadeiras_escolares])

{'CADEIRA ESCOLAR - C/ASSENTO E ENCOSTO EM POLIPROPILENO NA COR OVO, EM ESTRUTURA DE FERRO NA COR AZUL, C/PRNCHETA FRONTAL MOVEL REVESTIDA DE FORMICA, MED. 89CM DE ALTURA.',
 'CADEIRA ESCOLAR - C/ASSENTO E ENCOSTO EM POLIPROPILENO NA COR OVO, EM ESTRUTURA DE FERRO TUBULAR NA COR AZUL, C/PRANCHETA FRONTAL MOVEL REVESTIDA DE FORMICA, MED. 89CM DE ALTURA.',
 'CADEIRA ESCOLAR - ESTOFADA EM COURVIN PRETO, EM ESTRUTURA DE FERRO PRETO, C/BRACO MESA EM ARVOPLAC, C/SUP. P/LIVROS, MED. 80CM DE ALTURA.'}

In [97]:
cadeiras_escolares_ids = [ x['key'] for x in cadeiras_escolares ]

In [98]:
batch_write_update_medidas('items', cadeiras_escolares_ids, '89 x 76 x 50 (AxCxL)')

items: chunk 1 (29 documents)


## MESA REUNIÃO REDONDA

In [108]:
mesa_redonda = get_item_by_key(315370)

In [109]:
mesas_redondas = list(filter_list_of_dicts(data, 'short_descricao', mesa_redonda["short_descricao"]))

In [111]:
mesas_redondas_ids = [ x['key'] for x in mesas_redondas ]

In [112]:
batch_write_update_medidas('items', mesas_redondas_ids, '75 x 120 x 120 (AxCxL)')

items: chunk 1 (20 documents)


In [127]:
import subprocess
import shlex

comando = 'grep -P "(\d+) documents" AplicandoMedidas.ipynb'

resultado = subprocess.run(shlex.split(comando), stdout=subprocess.PIPE)

texto = resultado.stdout.decode('utf-8')

In [130]:
sum([int(x) for x in re.findall(r'\d+', texto)])

2418